In [6]:
import pandas as pd 
# Load the world cities dataset from a CSV file into a DataFrame
# The file path points to the location of the CSV file on the user's system
cities=pd.read_csv("C:/Users/aitma/Downloads/simplemaps_worldcities_basicv1.77/worldcities.csv")

In [7]:
cities=cities[cities['country']=='Morocco']

In [8]:
cities=cities[['city','lat','lng','admin_name']]

In [9]:
cities

,city,lat,lng,admin_name
214,Casablanca,33.5333,-7.5833,Casablanca-Settat
690,Fès,34.0433,-5.0033,Fès-Meknès
795,Marrakech,31.6300,-8.0089,Marrakech-Safi
809,Tangier,35.7767,-5.8039,Tanger-Tétouan-Al Hoceïma
904,Sale,34.0333,-6.8000,Rabat-Salé-Kénitra
...,...,...,...,...
46189,Lambarkiyine,33.2000,-7.5000,Casablanca-Settat
46604,Bir Anzarane,23.8918,-14.5364,Dakhla-Oued Ed-Dahab
46991,Tifariti,26.1580,-10.5670,Laâyoune-Sakia El Hamra
47293,Uad Damran,27.4181,-12.9842,Laâyoune-Sakia El Hamra


In [10]:
import requests
import time
import json

def fetch_data_with_error_handling(api_url, headers, params, retries=3, backoff_factor=2):
    """
    Fetch data from an API with error handling for rate limits, timeouts, and missing data.
    
    :param api_url: The API endpoint to fetch data from.
    :param headers: The headers to include in the request, such as API keys.
    :param params: The parameters to include in the request, such as latitude and longitude.
    :param retries: Number of retries if a request fails.
    :param backoff_factor: Time multiplier for exponential backoff.
    :return: JSON data if the request is successful, None otherwise.
    """
    for i in range(retries):
        try:
            response = requests.get(api_url, headers=headers, params=params)
            if response.status_code == 200:
                data = response.json()

                if not data or 'error' in data:
                    print("Warning: Data is missing or incomplete.")
                    return None

                return data

            elif response.status_code == 429:
                print("Rate limit exceeded. Retrying after backoff...")
                time.sleep(backoff_factor ** i)
                continue
            else:
                response.raise_for_status()  

        except requests.exceptions.HTTPError as e:
            print(f"HTTP error occurred: {e}")
        except requests.exceptions.ConnectionError as e:
            print(f"Connection error occurred: {e}")
        except requests.exceptions.Timeout as e:
            print(f"Timeout occurred: {e}")
        except requests.exceptions.RequestException as e:
            print(f"An error occurred: {e}")

        # Exponential backoff before retrying
        time.sleep(backoff_factor ** i)

    # Return None if all retries failed
    return None


    
url="https://api.meersens.com/environment/public/air/current"
headers={'apikey': 'jYxv9Yb5x1pTicfLRSwK29KaNPX1OAUl'}

json_data=[]
for index, row in cities.iterrows():
    params={"lat": row['lat'] , "lng": row['lng']}
    data = fetch_data_with_error_handling(url, headers, params)
    json_data.append(data)

print(json_data[0])


{'found': True, 'datetime': '2024-09-05T21:00:00.000Z', 'index': {'index_type': 'meersens', 'index_name': 'MAQI', 'qualification': 'Modéré', 'icon': None, 'color': '#ffdc00', 'description': "Le niveau de qualité de l'air présentera un risque pour la santé pour une durée d'exposition supérieure à plusieurs jours", 'value': 41.15, 'main_pollutants': ['o3']}, 'pollutants': {'co': {'shortcode': 'CO', 'name': 'Monoxyde de carbone', 'unit': 'µg/m³', 'found': True, 'value': 184.6, 'confidence': 5, 'index': {'index_type': 'meersens', 'index_name': 'MAQI', 'qualification': 'Parfait', 'icon': None, 'color': '#066b78', 'description': 'Le niveau de qualité de l’air ne présentera pas de risque pour la santé pour une durée d’exposition supérieure à plusieurs décennies', 'value': 0.45}}, 'no2': {'shortcode': 'NO2', 'name': "Dioxyde d'azote ", 'unit': 'µg/m³', 'found': True, 'value': 0, 'confidence': 5, 'index': {'index_type': 'meersens', 'index_name': 'MAQI', 'qualification': 'Parfait', 'icon': None,

In [11]:
# Define the file path
file_path = 'data.json'

# Write the list of dictionaries to a JSON file
with open(file_path, 'w') as file:
    json.dump(json_data, file, indent=4)

In [12]:
import json

# Function to recursively remove specified keys from JSON
def remove_keys(data, keys_to_remove):
    if isinstance(data, dict):
        # Remove the keys from the dictionary
        return {key: remove_keys(value, keys_to_remove) for key, value in data.items() if key not in keys_to_remove}
    elif isinstance(data, list):
        # Recursively handle lists
        return [remove_keys(item, keys_to_remove) for item in data]
    else:
        return data

with open('data.json', 'r') as file:
    data = json.load(file) 
    
# Keys to remove
keys_to_remove = ["found", "datetime", "icon", "color", "description"]

# Remove the keys
cleaned_data = remove_keys(data, keys_to_remove)

file_path = 'data.json'

# Write the list of dictionaries to a JSON file
with open(file_path, 'w') as file:
    json.dump(cleaned_data, file, indent=4)


In [13]:
data = cleaned_data

def check_null_values(data):
    # Flag to track if any null values are found
    null_found = False

    def check_value(value, key):
        nonlocal null_found
        if value is None:
            print(f"The value for '{key}' is null.")
            null_found = True

    def check_dict(d):
        for key, value in d.items():
            if isinstance(value, dict):
                check_dict(value)  # Recursively check nested dictionaries
            elif isinstance(value, (float, str)):
                check_value(value, key)

    # Start checking from the top level
    for item in data:
        check_dict(item)

    # Print message if no null values found
    if not null_found:
        print("No null float or string values found.")

# Step 3: Call the function
check_null_values(cleaned_data)

No null float or string values found.


In [14]:
file_path = 'data.json'

# Write the list of dictionaries to a JSON file
with open(file_path, 'w') as file:
    json.dump(cleaned_data, file, indent=4)

In [36]:
cities_data = []
indexes_data = []
pollutants_data = []

# Function to transform and extract data for each city
def transform_data(city_data):
  
    
    # Extract index data
    index_table = {
 
        "index_type": city_data['index']['index_type'],
        "index_name": city_data['index']['index_name'],
        "index_qualification": city_data['index']['qualification'],
        "index_value": city_data['index']['value']
    }
    
    # Extract pollutants data
    for key, pollutant in city_data['pollutants'].items():
        pollutant_row = {
        
            "shortcode": pollutant['shortcode'],
            "name": pollutant['name'],
            "unit": pollutant['unit'],
            "value": pollutant['value'],
            "confidence": pollutant['confidence'],
            "index_qualification": pollutant['index']['qualification'],
            "index_value": pollutant['index']['value']
        }
        pollutants_data.append(pollutant_row)
    
    # Append to the list
 
    indexes_data.append(index_table)

# Loop over each city's data in the JSON file
for city_data in cleaned_data:
    transform_data(city_data)

# Convert lists to DataFrames

index_df = pd.DataFrame(indexes_data)
pollutants_df = pd.DataFrame(pollutants_data)

In [37]:
pollutants_df

,shortcode,name,unit,value,confidence,index_qualification,index_value
0,CO,Monoxyde de carbone,µg/m³,184.6,5,Parfait,0.45
1,NO2,Dioxyde d'azote,µg/m³,0.0,5,Parfait,0.00
2,O3,Ozone,µg/m³,79.2,5,Modéré,41.15
3,PM10,Particules fines 10,µg/m³,15.2,5,Correct,30.83
4,PM2.5,Particules fines 2.5,µg/m³,6.1,5,Correct,33.62
...,...,...,...,...,...,...,...
3517,NO2,Dioxyde d'azote,µg/m³,0.0,5,Parfait,0.00
3518,O3,Ozone,µg/m³,83.3,5,Modéré,42.89
3519,PM10,Particules fines 10,µg/m³,141.7,5,Pauvre,72.81
3520,PM2.5,Particules fines 2.5,µg/m³,19.8,5,Moyen,51.63


In [38]:
index_df

,index_type,index_name,index_qualification,index_value
0,meersens,MAQI,Modéré,41.15
1,meersens,MAQI,Correct,38.65
2,meersens,MAQI,Moyen,58.59
3,meersens,MAQI,Modéré,45.02
4,meersens,MAQI,Modéré,43.11
...,...,...,...,...
582,meersens,MAQI,Correct,37.23
583,meersens,MAQI,Extrême,93.45
584,meersens,MAQI,Extrême,95.27
585,meersens,MAQI,Très pauvre,81.83


In [39]:
index_df.reset_index(drop=True, inplace=True)
cities.reset_index(drop=True, inplace=True)

# Concatenate both dataframes along the columns (axis=1)
index_df = pd.concat([index_df, cities['city']], axis=1)

In [40]:
index_df

,index_type,index_name,index_qualification,index_value,city
0,meersens,MAQI,Modéré,41.15,Casablanca
1,meersens,MAQI,Correct,38.65,Fès
2,meersens,MAQI,Moyen,58.59,Marrakech
3,meersens,MAQI,Modéré,45.02,Tangier
4,meersens,MAQI,Modéré,43.11,Sale
...,...,...,...,...,...
582,meersens,MAQI,Correct,37.23,Lambarkiyine
583,meersens,MAQI,Extrême,93.45,Bir Anzarane
584,meersens,MAQI,Extrême,95.27,Tifariti
585,meersens,MAQI,Très pauvre,81.83,Uad Damran


In [41]:
import pandas as pd

# Assuming cities dataframe has 587 cities and pollutants_df has 3522 rows
# Repeat each city name 6 times to match the 3522 rows in pollutants_df
repeated_cities = cities['city'].repeat(6).reset_index(drop=True)

# Make sure the length of repeated_cities matches pollutants_df
assert len(repeated_cities) == len(pollutants_df)

# Add the repeated city names as a new column in pollutants_df
pollutants_df['city'] = repeated_cities

# Inspect the result
pollutants_df


,shortcode,name,unit,value,confidence,index_qualification,index_value,city
0,CO,Monoxyde de carbone,µg/m³,184.6,5,Parfait,0.45,Casablanca
1,NO2,Dioxyde d'azote,µg/m³,0.0,5,Parfait,0.00,Casablanca
2,O3,Ozone,µg/m³,79.2,5,Modéré,41.15,Casablanca
3,PM10,Particules fines 10,µg/m³,15.2,5,Correct,30.83,Casablanca
4,PM2.5,Particules fines 2.5,µg/m³,6.1,5,Correct,33.62,Casablanca
...,...,...,...,...,...,...,...,...
3517,NO2,Dioxyde d'azote,µg/m³,0.0,5,Parfait,0.00,Lemsid
3518,O3,Ozone,µg/m³,83.3,5,Modéré,42.89,Lemsid
3519,PM10,Particules fines 10,µg/m³,141.7,5,Pauvre,72.81,Lemsid
3520,PM2.5,Particules fines 2.5,µg/m³,19.8,5,Moyen,51.63,Lemsid


In [42]:
cities

,city,lat,lng,admin_name,city_id
0,Casablanca,33.5333,-7.5833,Casablanca-Settat,1
1,Fès,34.0433,-5.0033,Fès-Meknès,2
2,Marrakech,31.6300,-8.0089,Marrakech-Safi,3
3,Tangier,35.7767,-5.8039,Tanger-Tétouan-Al Hoceïma,4
4,Sale,34.0333,-6.8000,Rabat-Salé-Kénitra,5
...,...,...,...,...,...
582,Lambarkiyine,33.2000,-7.5000,Casablanca-Settat,583
583,Bir Anzarane,23.8918,-14.5364,Dakhla-Oued Ed-Dahab,584
584,Tifariti,26.1580,-10.5670,Laâyoune-Sakia El Hamra,585
585,Uad Damran,27.4181,-12.9842,Laâyoune-Sakia El Hamra,586


In [43]:
import pandas as pd
import numpy as np

import pandas as pd

def fill_nulls(df):
    # Drop rows where 'city' column has null values
    if 'city' in df.columns:
        df = df.dropna(subset=['city'])
    
    # Iterate over each column in the DataFrame
    for column in df.columns:
        if pd.api.types.is_numeric_dtype(df[column]):
            # If the column is numerical, fill null values with the median
            median_value = df[column].median()
            df[column].fillna(median_value, inplace=True)
        elif pd.api.types.is_categorical_dtype(df[column]) or pd.api.types.is_object_dtype(df[column]):
            # If the column is categorical (including object type), fill null values with the mode
            mode_value = df[column].mode()[0]  # mode() returns a Series
            df[column].fillna(mode_value, inplace=True)

    return df


index_df=fill_nulls(index_df)
pollutants_df=fill_nulls(pollutants_df)
cities=fill_nulls(cities)


C:\Users\aitma\AppData\Local\Temp\ipykernel_7212\2974099525.py:17: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  elif pd.api.types.is_categorical_dtype(df[column]) or pd.api.types.is_object_dtype(df[column]):
C:\Users\aitma\AppData\Local\Temp\ipykernel_7212\2974099525.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(mode_value, inplace=True)
C:\Users\aitma\AppData\Local\Temp\ipykernel_7212\2974099525.py:

In [44]:
index_df

,index_type,index_name,index_qualification,index_value,city
0,meersens,MAQI,Modéré,41.15,Casablanca
1,meersens,MAQI,Correct,38.65,Fès
2,meersens,MAQI,Moyen,58.59,Marrakech
3,meersens,MAQI,Modéré,45.02,Tangier
4,meersens,MAQI,Modéré,43.11,Sale
...,...,...,...,...,...
582,meersens,MAQI,Correct,37.23,Lambarkiyine
583,meersens,MAQI,Extrême,93.45,Bir Anzarane
584,meersens,MAQI,Extrême,95.27,Tifariti
585,meersens,MAQI,Très pauvre,81.83,Uad Damran


In [45]:
pollutants_df

,shortcode,name,unit,value,confidence,index_qualification,index_value,city
0,CO,Monoxyde de carbone,µg/m³,184.6,5,Parfait,0.45,Casablanca
1,NO2,Dioxyde d'azote,µg/m³,0.0,5,Parfait,0.00,Casablanca
2,O3,Ozone,µg/m³,79.2,5,Modéré,41.15,Casablanca
3,PM10,Particules fines 10,µg/m³,15.2,5,Correct,30.83,Casablanca
4,PM2.5,Particules fines 2.5,µg/m³,6.1,5,Correct,33.62,Casablanca
...,...,...,...,...,...,...,...,...
3517,NO2,Dioxyde d'azote,µg/m³,0.0,5,Parfait,0.00,Lemsid
3518,O3,Ozone,µg/m³,83.3,5,Modéré,42.89,Lemsid
3519,PM10,Particules fines 10,µg/m³,141.7,5,Pauvre,72.81,Lemsid
3520,PM2.5,Particules fines 2.5,µg/m³,19.8,5,Moyen,51.63,Lemsid


In [46]:
cities

,city,lat,lng,admin_name,city_id
0,Casablanca,33.5333,-7.5833,Casablanca-Settat,1
1,Fès,34.0433,-5.0033,Fès-Meknès,2
2,Marrakech,31.6300,-8.0089,Marrakech-Safi,3
3,Tangier,35.7767,-5.8039,Tanger-Tétouan-Al Hoceïma,4
4,Sale,34.0333,-6.8000,Rabat-Salé-Kénitra,5
...,...,...,...,...,...
582,Lambarkiyine,33.2000,-7.5000,Casablanca-Settat,583
583,Bir Anzarane,23.8918,-14.5364,Dakhla-Oued Ed-Dahab,584
584,Tifariti,26.1580,-10.5670,Laâyoune-Sakia El Hamra,585
585,Uad Damran,27.4181,-12.9842,Laâyoune-Sakia El Hamra,586


In [47]:
import pandas as pd

# Calculate total value for each city
total_values = pollutants_df.groupby('city')['value'].sum().reset_index()
total_values.rename(columns={'value': 'total_value'}, inplace=True)

# Merge total values with the original DataFrame
pollutants_df = pollutants_df.merge(total_values, on='city', how='left')

pollutants_df


,shortcode,name,unit,value,confidence,index_qualification,index_value,city,total_value
0,CO,Monoxyde de carbone,µg/m³,184.6,5,Parfait,0.45,Casablanca,292.5
1,NO2,Dioxyde d'azote,µg/m³,0.0,5,Parfait,0.00,Casablanca,292.5
2,O3,Ozone,µg/m³,79.2,5,Modéré,41.15,Casablanca,292.5
3,PM10,Particules fines 10,µg/m³,15.2,5,Correct,30.83,Casablanca,292.5
4,PM2.5,Particules fines 2.5,µg/m³,6.1,5,Correct,33.62,Casablanca,292.5
...,...,...,...,...,...,...,...,...,...
3517,NO2,Dioxyde d'azote,µg/m³,0.0,5,Parfait,0.00,Lemsid,399.9
3518,O3,Ozone,µg/m³,83.3,5,Modéré,42.89,Lemsid,399.9
3519,PM10,Particules fines 10,µg/m³,141.7,5,Pauvre,72.81,Lemsid,399.9
3520,PM2.5,Particules fines 2.5,µg/m³,19.8,5,Moyen,51.63,Lemsid,399.9


In [48]:
cities['city_id'] = range(1, len(cities) + 1)
city_id_map = dict(zip(cities['city'], cities['city_id']))

# Replace the city names in index_df and pollutants_df with city_id
index_df['city_id'] = index_df['city'].map(city_id_map)
index_df.drop('city', axis=1, inplace=True)  # Remove the city column as it's replaced by city_id

pollutants_df['city_id'] = pollutants_df['city'].map(city_id_map)
pollutants_df.drop('city', axis=1, inplace=True)  

In [49]:
index_df

,index_type,index_name,index_qualification,index_value,city_id
0,meersens,MAQI,Modéré,41.15,1
1,meersens,MAQI,Correct,38.65,2
2,meersens,MAQI,Moyen,58.59,3
3,meersens,MAQI,Modéré,45.02,4
4,meersens,MAQI,Modéré,43.11,5
...,...,...,...,...,...
582,meersens,MAQI,Correct,37.23,583
583,meersens,MAQI,Extrême,93.45,584
584,meersens,MAQI,Extrême,95.27,585
585,meersens,MAQI,Très pauvre,81.83,586


In [50]:
pollutants_df

,shortcode,name,unit,value,confidence,index_qualification,index_value,total_value,city_id
0,CO,Monoxyde de carbone,µg/m³,184.6,5,Parfait,0.45,292.5,1
1,NO2,Dioxyde d'azote,µg/m³,0.0,5,Parfait,0.00,292.5,1
2,O3,Ozone,µg/m³,79.2,5,Modéré,41.15,292.5,1
3,PM10,Particules fines 10,µg/m³,15.2,5,Correct,30.83,292.5,1
4,PM2.5,Particules fines 2.5,µg/m³,6.1,5,Correct,33.62,292.5,1
...,...,...,...,...,...,...,...,...,...
3517,NO2,Dioxyde d'azote,µg/m³,0.0,5,Parfait,0.00,399.9,587
3518,O3,Ozone,µg/m³,83.3,5,Modéré,42.89,399.9,587
3519,PM10,Particules fines 10,µg/m³,141.7,5,Pauvre,72.81,399.9,587
3520,PM2.5,Particules fines 2.5,µg/m³,19.8,5,Moyen,51.63,399.9,587


In [51]:
cities

,city,lat,lng,admin_name,city_id
0,Casablanca,33.5333,-7.5833,Casablanca-Settat,1
1,Fès,34.0433,-5.0033,Fès-Meknès,2
2,Marrakech,31.6300,-8.0089,Marrakech-Safi,3
3,Tangier,35.7767,-5.8039,Tanger-Tétouan-Al Hoceïma,4
4,Sale,34.0333,-6.8000,Rabat-Salé-Kénitra,5
...,...,...,...,...,...
582,Lambarkiyine,33.2000,-7.5000,Casablanca-Settat,583
583,Bir Anzarane,23.8918,-14.5364,Dakhla-Oued Ed-Dahab,584
584,Tifariti,26.1580,-10.5670,Laâyoune-Sakia El Hamra,585
585,Uad Damran,27.4181,-12.9842,Laâyoune-Sakia El Hamra,586


In [52]:
cities.drop(columns=['city_id'], inplace=True)

In [53]:
cities

,city,lat,lng,admin_name
0,Casablanca,33.5333,-7.5833,Casablanca-Settat
1,Fès,34.0433,-5.0033,Fès-Meknès
2,Marrakech,31.6300,-8.0089,Marrakech-Safi
3,Tangier,35.7767,-5.8039,Tanger-Tétouan-Al Hoceïma
4,Sale,34.0333,-6.8000,Rabat-Salé-Kénitra
...,...,...,...,...
582,Lambarkiyine,33.2000,-7.5000,Casablanca-Settat
583,Bir Anzarane,23.8918,-14.5364,Dakhla-Oued Ed-Dahab
584,Tifariti,26.1580,-10.5670,Laâyoune-Sakia El Hamra
585,Uad Damran,27.4181,-12.9842,Laâyoune-Sakia El Hamra


In [54]:
cities.to_csv('cities.csv', index=False, header=True, sep=',')
index_df.to_csv('index_df.csv', index=False, header=True, sep=',')
pollutants_df.to_csv('pollutants_df.csv', index=False, header=True, sep=',')


In [69]:
import mysql.connector
import pandas as pd

# Connect to MySQL
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='Qwerty1234/',
    database='new_schema'
)
cursor = conn.cursor()

# Load the DataFrames
cities_df = pd.read_csv('cities.csv')
index_df = pd.read_csv('index_df.csv')
pollutants_df = pd.read_csv('pollutants_df.csv')



# Option 2: Reset AUTO_INCREMENT value manually (if you want to keep data)
cursor.execute("ALTER TABLE cities AUTO_INCREMENT = 1")
# Insert cities and create city_id map
city_id_map = {}
insert_cities_query = """
INSERT INTO cities (city, lat, lng, admin_name)
VALUES (%s, %s, %s, %s)
ON DUPLICATE KEY UPDATE lat=VALUES(lat), lng=VALUES(lng), admin_name=VALUES(admin_name)
"""
for index, row in cities_df.iterrows():
    cursor.execute(insert_cities_query, (row['city'], row['lat'], row['lng'], row['admin_name']))

# Retrieve city_id for the inserted cities
cursor.execute("SELECT city, city_id FROM cities")
city_id_map = dict(cursor.fetchall())

# Insert index data
insert_index_query = """
INSERT INTO index_table (index_type, index_name, index_qualification, index_value, city_id)
VALUES (%s, %s, %s, %s, %s)
"""
index_data = [(row['index_type'], row['index_name'], row['index_qualification'], row['index_value'], city_id_map.get(row['city'])) for _, row in index_df.iterrows()]
cursor.executemany(insert_index_query, index_data)

# Insert pollutants data
insert_pollutants_query = """
INSERT INTO pollutants (shortcode, name, unit, value, confidence, index_qualification, index_value, city_id)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""
pollutants_data = [(row['shortcode'], row['name'], row['unit'], row['value'], row['confidence'], row['index_qualification'], row['index_value'], city_id_map.get(row['city'])) for _, row in pollutants_df.iterrows()]
cursor.executemany(insert_pollutants_query, pollutants_data)

# Commit the transactions and close the connection
conn.commit()
cursor.close()
conn.close()
